In [16]:
import pymupdf
import json
from tqdm import tqdm
from openai import OpenAI
import copy
import re
import io
import os
import sys
import time

import fitz
from pdf2image import convert_from_path
from PIL import Image

In [17]:
BOOK_PATH = "../../files/data-science_book/data-science.pdf"
TEX_PATH = "../../files/data-science_book/outputs/data-science_pg_sep_bib.tex"
# IMG_DIR = "/content/drive/My Drive/pdf2latex/new_approach_test/images"  # found images are stored in this subfolder
OUTPUT_TEX_FILE = "../../files/data-science_book/outputs/data-science_cleaned_2p__toc.tex"

# TOC_PATH  = "../../files/data-science_book/data-science_toc.pdf"
# toc = pymupdf.open(TOC_PATH)
# all_toc_text = ""
# for page_num in range(len(toc)):
#     page = toc[page_num]
#     text = page.get_text()
#     all_toc_text += f"\n--- Page {page_num + 1} ---\n"
#     all_toc_text += text.strip() + "\n"


toc = pymupdf.open(BOOK_PATH)
table_of_contents  = toc.get_toc()


[[1, 'Preface', 6],
 [1, 'Contents', 12],
 [1, '1\rWhat is Data Science?', 19],
 [2, '1.1 Computer Science, Data Science, and Real', 20],
 [2, 'Science', 20],
 [2, '1.2 Asking Interesting Questions from Data', 22],
 [3, '1.2.1 The Baseball Encyclopedia', 23],
 [3, '1.2.2 The Internet Movie Database (IMDb)', 25],
 [3, '1.2.3 Google Ngrams', 28],
 [3, '1.2.4 New York Taxi Records', 29],
 [2, '1.3 Properties of Data', 32],
 [3, '1.3.1 Structured vs. Unstructured Data', 32],
 [3, '1.3.2 Quantitative vs. Categorical Data', 33],
 [3, '1.3.3 Big Data vs. Little Data', 33],
 [2, '1.4 Classi\x0ccation and Regression', 34],
 [2, '1.5 Data Science Television: The Quant Shop', 35],
 [3, '1.5.1 Kaggle Challenges', 37],
 [2, '1.6 About the War Stories', 37],
 [2, '1.7 War Story: Answering the Right Question', 39],
 [2, '1.8 Chapter Notes', 40],
 [2, '1.9 Exercises', 41],
 [1, '2\rMathematical Preliminaries', 44],
 [2, '2.1 Probability', 44],
 [3, '2.1.1 Probability vs. Statistics', 46],
 [3, '2.1.2 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")

In [ ]:
OPENAI_API_KEY = api_key
client = OpenAI(api_key = OPENAI_API_KEY)

In [4]:
doc = pymupdf.open(BOOK_PATH)
doc.page_count

456

In [5]:
with open(TEX_PATH, 'r') as file:
    tex_file_contents = file.read()

In [6]:
def get_page_text_data(page_number, span_counter, text_data, doc):
    page = doc[page_number]
    # print(page)

    # Read page text as a dictionary, suppressing extra spaces in CJK fonts
    blocks = page.get_text("dict", flags=0)["blocks"]
    # print(blocks)
    line_number_in_page = 0
    span_number_in_page = 0
    # print("--Old--")
    for block_number, b in enumerate(blocks):  # Iterate through the text blocks
        span_number_in_block = 0  # Initialize span counter for the block

        # print(b["lines"])

        for l in b["lines"]:  # Iterate through the text lines
            # print(l)
            line_number_in_page += 1
            span_number_in_line = 0  # Initialize span counter for the line
            # print("Spans : "+ str(len(l["spans"])))
            for s in l["spans"]:  # Iterate through the text spans
                 # Create a deep copy of the original span dictionary to preserve all its properties
                span_data = copy.deepcopy(s)

                # Temporary removal to check hwo it works
                del span_data["size"]
                # del span_data["flags"]
                del span_data["bidi"]
                del span_data["char_flags"]
                del span_data["ascender"]
                del span_data["descender"]
                del span_data['origin']
                del span_data['bbox']
                del span_data['color']
                del span_data['font']




                # Add additional properties if needed
                # span_data["page_number"] = page_number
                # span_data["span_number_overall"] = span_counter
                # span_data["span_number_in_line"] = span_number_in_line
                # span_data["span_number_in_block"] = span_number_in_block
                # span_data["span_number_in_page"] = span_number_in_page
                # span_data["block_number"] = block_number

                # Extract and store bounding box information
                # x0, y0, x1, y1 = span_data["bbox"]
                # span_data["indent_left"] = x0
                # span_data["indent_top"] = y0
                # span_data["x1"] = x1
                # span_data["y1"] = y1

                # Decompose flags to determine font styles
                decomposed_flags = flags_decomposer(span_data["flags"])
                span_data["is_italic"] = "italic" in decomposed_flags
                span_data["is_bold"] = "bold" in decomposed_flags
                span_data["is_superscript"] = "superscript" in decomposed_flags

                del span_data["flags"]

                # Append the dictionary to the text_data list
                text_data.append(span_data)
                # Increase the overall counters
                span_counter += 1
                span_number_in_line += 1  # Increase the span counter within the line
                span_number_in_block += 1  # Increase the span counter within the block
                span_number_in_page += 1
    # print("---Old End---")
    return text_data, span_counter

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

In [7]:
text_data = []
span_counter = 0
for i in range(16, 16+16):
  text_data, span_counter = get_page_text_data(i, span_counter, text_data, doc)
  # text_data, span_counter = get_page_text_data(i, span_counter, doc)

print("Length : " + str(len(text_data)))

Length : 772


In [8]:
page_breaks = re.findall(r'%---- Page End Break Here ---- Page : (\d+)', tex_file_contents)
page_positions = {int(page): pos.start() for page, pos in zip(page_breaks, re.finditer(r'%---- Page End Break Here ---- Page : \d+', tex_file_contents))}


In [19]:
first_page_command = """
You will receive an unformatted LaTeX (.tex) file part of a book along with a separate JSON file containing formatting instructions.  
Your task is to format the LaTeX file part according to the JSON data while ensuring proper structure and presentation for a book.  

### **Formatting Guidelines:**  

**1. Apply JSON Formatting Instructions:**  
   - Modify only the necessary parts based on JSON data.  
   - Do **not** make arbitrary changes—only apply specified formatting corrections.  

**2. Book Structure:**  
   - The given tex file considers the document as an article, but you need to treat it as a book in the imports section as well as the actual data.
   - Add a usepackage command for indexing, makeidx and \makeindex in the import section
   - Generate table of contents dynamically.
   - Organize content into proper **chapters, sections, and subsections** 
   - **Do not assume chapter starts based on recurring text** (e.g., headers repeated on every page).  
   - If chapter names and numbers appear on every page in the JSON, **ignore them** when determining chapter breaks.  
   - **Remove hardcoded numbering** for chapters and sections, allowing LaTeX to handle it automatically.  
   - Make the Contents Page dynamically if contents is present in the .tex file part. Do not hardcode the table of contents.

**3. Image Handling:**  
   - Convert all instances of `\includegraphics{}` into a proper `figure` environment:  

**4. Table Formatting:**  
   - Ensure tables are properly structured with appropriate spacing, alignment, and captions for readability.  

**5. Italics Handling:**  
   - Apply italics **only** to content explicitly marked as italicized in the JSON data.  

**6. Document Setup:**  
   - This is the **first part of the book**, so include **all necessary LaTeX imports and the document class**.  
   - **Do not modify LaTeX package imports unless explicitly required in the JSON file.** 
   - Do **not** manually start or end the document unless such commands are explicitly present.  

**7. Strict Output Requirements:**  
   - The output **must be pure LaTeX code**—**no explanations, comments, or markdown syntax.**  
   - The formatted output will be **directly appended** to the `.tex` file, so it must be immediately compilable.  

**8. Accuracy and Consistency:**  
   - Since the book is processed in parts, formatting should be **consistent across all sections**.  
   - **Do not introduce new formatting styles** that conflict with previous or upcoming sections.  
   - Ensure that all content is preserved and formatted correctly—no missing text, no misinterpretations.  

**Final Note:**  
Errors in formatting can **significantly affect the compiled document.** Ensure precise execution of all instructions while preserving the document's original meaning and intent.  
  
"""



next_pages_prompt = """
You will receive a portion of a LaTeX (.tex) file part of a book along with a separate JSON file containing formatting instructions.  
Your task is to format this LaTeX file part according to the provided JSON data while maintaining consistency with previous sections.  

### **Formatting Guidelines:**  

**1. Apply JSON Formatting Instructions:**  
   - Modify only the necessary parts as specified in the JSON data.  
   - Do **not** assume formatting—only apply explicit corrections.  

**2. Maintain Book Structure:**
   - The given tex file considers the document as an article, but you need to treat it as a book in the actual data.
   - Organize content into proper **chapters, sections, and subsections** only if explicitly marked in the `.tex` file.  
   - **Do not assume chapter starts based on recurring text** (e.g., headers repeated on every page).  
   - If chapter names and numbers appear on every page in the JSON, **ignore them** when determining chapter breaks.  
   - **Remove hardcoded numbering** on chapters, sections and subsections and rely on LaTeX’s automatic numbering system strictly.
   - Dont use * tags like \section*{section name} as they remove the latex numbering system.  
   - Make the Contents Page dynamically if contents is present in the .tex file part. Do not hardcode the table of contents.
**3. Image Handling:**  
   - Convert `\includegraphics{}` into a properly formatted `figure` environment:  


**4. Table Formatting:**  
   - Ensure tables are properly structured, aligned, and formatted for readability.  

**5. Italics Handling:**  
   - Apply italics **only** to content explicitly marked as italicized in the JSON data.  

**6. Document Integrity:**  
   - **Do not add any LaTeX preamble, document class, or import statements.**  
   - **Do not modify LaTeX package imports unless explicitly required in the JSON file.** 
   - **Do not include `\begin{document}` or `\end{document}`** unless explicitly present in the provided `.tex` file.  

**7. Strict Output Requirements:**  
   - The output **must be pure LaTeX code**—no explanations, comments, or markdown syntax.  
   - The formatted output will be **directly appended** to an existing `.tex` file, so it must be immediately compilable.  

**8. Accuracy and Consistency:**  
   - Ensure formatting is **consistent with previous sections** of the book.  
   - **Do not introduce new formatting styles** that conflict with earlier parts.  
   - Ensure **all content is retained**, formatted correctly, and adheres to the document’s original intent.  

**Final Note:**  
Errors in formatting can **significantly impact the final compiled document.** Follow the instructions precisely to maintain a high-quality, structured LaTeX book.  

"""


"""
     ```
     \begin{figure}[h]
         \centering
         \includegraphics{filename}
         \caption{Caption text}
         \label{fig:label}
     \end{figure}
     ```  
"""


'\n     ```\n     \x08egin{figure}[h]\n         \\centering\n         \\includegraphics{filename}\n         \\caption{Caption text}\n         \\label{fig:label}\n     \\end{figure}\n     ```  \n'

In [10]:
def generate_response(command, data, prev_response, temperature=1):
  first_page_prompt = f"{command} \n {data}"
  default_page_prompt = f"""{command} \n{data}"""
  prompt_content = first_page_prompt if prev_response == "" else default_page_prompt
  response =  client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant. You convert PDF documents to LaTeX."},
    {"role": "user", "content": f"{prompt_content}"}
    ],
  temperature=temperature
  )
  return response.choices[0].message.content

In [11]:
book_page_data = {}
page_numbers = []
for i in range(len(doc)):
    page = doc[i]
    page_numbers.append(page.get_label())
    book_page_data[i] = page.get_text("text").replace("\n", " ")

In [12]:
def get_pages_data(start_indx, end_indx, doc):
    text_data = []
    span_counter = 0
    for i in range(start_indx, end_indx+1):
        text_data, span_counter = get_page_text_data(i, span_counter, text_data, doc)
    # text_data, span_counter = get_page_text_data(i, span_counter, doc)

    # print("Length : " + str(len(text_data)))
    return text_data

In [72]:
start_indx = 0

tex_start_pos = 0
tex_end_pos = 0

first_part = 1

parts = len(page_breaks)
counter = 1
for page in tqdm(page_breaks[:parts]):
    
    end_indx = page_numbers.index(page)
    text_data = get_pages_data(start_indx, end_indx, doc)

    tex_end_pos = page_positions[int(page)]
    tex_contents = tex_file_contents[tex_start_pos:tex_end_pos]

    # gpt api call
    combined_data = (
    "Below is pre-generated TeX code without proper formatting.\n\n"
    f"{tex_contents}\n\n"
    "Below is the JSON  data which contains formatting :\n\n"
    f"{text_data}"
    )
    if counter == parts:
      combined_data += "\n\n"
      combined_data += "This was the last part, close the latex document with end document. Before that, make an index using \makeindex command and similarly make a bibliography."
    else:
       combined_data += "\n\n"
       combined_data += f"This is the {counter} part of the book, do not close the latex document with end document."

    command = first_page_command if first_part==1 else next_pages_prompt
    response = generate_response(combined_data, command, "") # reversed the combined_data and command
        
    first_part = 0
    counter+=1
    with open(OUTPUT_TEX_FILE, 'a') as f:
      f.write(response + "\n")
      f.write(f"%---- Page End Break Here ---- Page : {page}\n")


    # update positions
    tex_start_pos = tex_end_pos+1
    start_indx = end_indx+1


100%|██████████| 323/323 [53:02<00:00,  9.85s/it]  


In [ ]:
# final till the end check

def remove_latex_and_ticks(text):
    return re.sub(r'```latex|```', '', text)

start_indx = 0

tex_start_pos = 0
tex_end_pos = 0

first_part = 1

parts = len(page_breaks)
counter = 1

skipper = 1
for page in tqdm(page_breaks[:parts]):
    
    if skipper != 2:
       skipper += 1
       continue

    skipper = 1
    
    end_indx = page_numbers.index(page)
    text_data = get_pages_data(start_indx, end_indx, doc)

    tex_end_pos = page_positions[int(page)]
    tex_contents = tex_file_contents[tex_start_pos:tex_end_pos]

    # gpt api call
    combined_data = (
    "Below is pre-generated TeX code without proper formatting.\n\n"
    f"{tex_contents}\n\n"
    "Below is the JSON  data which contains formatting :\n\n"
    f"{text_data}"
    )
    if counter == parts:
      combined_data += "\n\n"
      combined_data += "This was the last part, close the latex document with end document. Before that, make an index using \makeindex command and similarly make a bibliography."
    else:
       combined_data += "\n\n"
       combined_data += f"This is the {counter} part of the book, do not close the latex document with end document."

    command = first_page_command if first_part==1 else next_pages_prompt
    response = generate_response(combined_data, command, "") # reversed the combined_data and command
    response = remove_latex_and_ticks(response)
    first_part = 0
    counter+=1

    with open(OUTPUT_TEX_FILE, 'a') as f:
      f.write(response + "\n")
      f.write(f"%---- Page End Break Here ---- Page : {page}\n")


    # update positions
    tex_start_pos = tex_end_pos+1
    start_indx = end_indx+1


 21%|██▏       | 70/327 [08:55<25:54,  6.05s/it] 

In [ ]:
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Function to remove unwanted LaTeX ticks
def remove_latex_and_ticks(text):
    return re.sub(r'```latex|```', '', text)

# Function to handle API call
def process_part(index, start_indx, end_indx, tex_start_pos, tex_end_pos, counter, first_part, parts, page, doc):
    text_data = get_pages_data(start_indx, end_indx, doc)
    tex_contents = tex_file_contents[tex_start_pos:tex_end_pos]

    # Construct the combined data for API call
    combined_data = (
        "Below is pre-generated TeX code without proper formatting.\n\n"
        f"{tex_contents}\n\n"
        "Below is the JSON data which contains formatting:\n\n"
        f"{text_data}"
    )
    
    if counter == parts-1:
        combined_data += "\n\nThis was the last part, close the LaTeX document with end document. Before that, make an index using \\makeindex command and similarly make a bibliography."
    else:
        combined_data += f"\n\nThis is the {counter} part of the book, do not close the LaTeX document with end document"

    command = first_page_command if first_part == 1 else next_pages_prompt
    response = generate_response(combined_data, command, "")  # API call
    response = remove_latex_and_ticks(response)

    return index, response, page  # Return index for ordering

# Parallel processing with batching
start_indx = 0
tex_start_pos = 0
first_part = 1
parts = len(page_breaks)
counter = 1
skipper = 1

responses_dict = {}  # Store responses by index
batch_size = 200  # Number of parallel requests at a time

with ThreadPoolExecutor(max_workers=batch_size) as executor:  
    futures_list = []  # Store pending API calls

    for idx, page in enumerate(tqdm(page_breaks[:parts])):
        if skipper != 2:
            skipper += 1
            continue
        skipper = 1

        end_indx = page_numbers.index(page)
        tex_end_pos = page_positions[int(page)]

        # Submit task and add to batch
        future = executor.submit(process_part, idx, start_indx, end_indx, tex_start_pos, tex_end_pos, counter, first_part, parts, page, doc)
        futures_list.append(future)

        # Update positions
        tex_start_pos = tex_end_pos + 1
        start_indx = end_indx + 1
        first_part = 0
        counter += 1

        # Process in batches of 5
        if len(futures_list) >= batch_size:
            for future in tqdm(as_completed(futures_list)):  # Wait for all in batch
                index, response, page = future.result()
                responses_dict[index] = (response, page)
            futures_list = []  # Clear batch before next set

            # Optional: Small delay to avoid rate limits (modify as per API rules)
            time.sleep(5)

    # Process remaining pending requests
    for future in as_completed(futures_list):
        index, response, page = future.result()
        responses_dict[index] = (response, page)

# Write responses in order
with open(OUTPUT_TEX_FILE, 'a') as f:
    for index in sorted(responses_dict.keys()):  # Ensure ordered writing
        response, page = responses_dict[index]
        f.write(response + "\n")
        f.write(f"%---- Page End Break Here ---- Page : {page}\n")


100%|██████████| 327/327 [00:02<00:00, 130.84it/s]
